In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
#getting the data
data_file_path = './data'
file_list = os.listdir(data_file_path)

swda_df = pd.concat([pd.read_csv(data_file_path+'/'+f) for f in file_list ], ignore_index=True)

display(swda_df)

print("Shape of the Data : ", swda_df.shape)

,swda_filename,ptb_basename,conversation_no,transcript_index,act_tag,caller,utterance_index,subutterance_index,text,pos,trees,ptb_treenumbers
0,sw07utt/sw_0763_3402.utt,3/sw3402,3402,0,o,A,1,1,All right. /,All/RB right/JJ ./.,NaN,NaN
1,sw07utt/sw_0763_3402.utt,3/sw3402,3402,1,sd^t,A,1,2,I guess tonight's subject is to discuss curre...,I/PRP guess/VBP tonight/RB 's/POS subject/NN i...,NaN,NaN
2,sw07utt/sw_0763_3402.utt,3/sw3402,3402,2,aa,B,2,1,Right. /,Right/UH ./.,NaN,NaN
3,sw07utt/sw_0763_3402.utt,3/sw3402,3402,3,+,A,3,1,"-- satisfied [ with, + with ] the, {F uh } --","--/: satisfied/JJ with/IN ,/, with/IN the/DT ,...",NaN,NaN
4,sw07utt/sw_0763_3402.utt,3/sw3402,3402,4,%,B,4,1,"That's, -/","That/DT 's/BES ,/,",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
223601,sw02utt/sw_0259_2020.utt,2/sw2020,2020,264,%,B,134,6,I - /,[ I/PRP ],(S-UNF (NP-SBJ (PRP I)) (-DFL- N_S)),379
223602,sw02utt/sw_0259_2020.utt,2/sw2020,2020,265,%,B,134,7,"me, - /","[ me/PRP ] ,/,","(NP (PRP me) (, ,) (-DFL- N_S))",380
223603,sw02utt/sw_0259_2020.utt,2/sw2020,2020,266,sv,B,134,8,"{C and } you never would think [ that they, + ...",and/CC [ you/PRP ] never/RB would/MD think/VB...,(S (CC and) (NP-SBJ (PRP you)) (ADVP-TMP (RB n...,381
223604,sw02utt/sw_0259_2020.utt,2/sw2020,2020,267,%,B,134,9,"{C but, } -/","but/CC ,/,","(S-UNF (CC but) (, ,) (-DFL- N_S))",382


Shape of the Data :  (223606, 12)


In [4]:
#Processing the dataset. 

#Experiment on POS, tree and ptb_treenumbers as a feature
parameters = ['utterance_index', 'subutterance_index', 'text', 'act_tag']
swda_df = swda_df[parameters]

# Creating a new column named is_backchannel
# Marking the column as 1 if act_tag is b else 0
is_backchannel = np.where(swda_df.act_tag == 'b', 1, 0)
swda_df['is_backchannel'] = is_backchannel

display(swda_df)


,utterance_index,subutterance_index,text,act_tag,is_backchannel
0,1,1,All right. /,o,0
1,1,2,I guess tonight's subject is to discuss curre...,sd^t,0
2,2,1,Right. /,aa,0
3,3,1,"-- satisfied [ with, + with ] the, {F uh } --",+,0
4,4,1,"That's, -/",%,0
...,...,...,...,...,...
223601,134,6,I - /,%,0
223602,134,7,"me, - /",%,0
223603,134,8,"{C and } you never would think [ that they, + ...",sv,0
223604,134,9,"{C but, } -/",%,0
